# Reshaping in Pandas

- https://pandas.pydata.org/pandas-docs/stable/reshaping.html
- http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/
- https://github.com/guipsamora/pandas_exercises

In [1]:
from collections import OrderedDict
from pandas import DataFrame
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
d = DataFrame(table)
d

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€


## Pivot

Pivot takes 3 arguements with the following names: `index`, `columns`, and `values`.

In [3]:
d.pivot(index='Item', columns='CType', values='USD')

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$


If the `values` argument is omitted, and the input DataFrame has __more than one column of values__ which are not used as column or index inputs to pivot, then the resulting “pivoted” DataFrame will have __hierarchical__ columns whose topmost level indicates the respective value column:

In [4]:
d.pivot(index='Item', columns='CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0     2$   1$    NaN     2€   1€    NaN
Item1    NaN   3$     4$    NaN   3€     4€

![pivoting_simple1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple1.png)

<img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_pivot.png" width="640">

### Error pivot with duplicated index combination

The `pivot()` method takes at least 2 column names as parameters - the `index` and the `columns`. When we have multiple rows with the __same values__ for these columns, the pivot method can not know what should be the value of the corresponding value in the pivoted table. Thus, it throws an exception

Therefore, before calling pivot we need to ensure that our data does not have rows with duplicate values for the specified columns. If we can’t ensure this we may have to use the pivot_table method instead.

In [5]:
columns = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$',  '2$',  '3$',  '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
d2 = DataFrame(columns)
print(d2)

try:
    p = d2.pivot(index='Item', columns='CType', values='USD')
except ValueError as e:
    print(f'\nException of type {type(e).__name__}:\n{e.args!r}')

    Item   CType USD  EU
0  Item0    Gold  1$  1€
1  Item0  Bronze  2$  2€
2  Item0    Gold  3$  3€
3  Item1  Silver  4$  4€

Exception of type ValueError:
('Index contains duplicate entries, cannot reshape',)


![](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple_error.png)

### Pivoting By Multiple Values

When omit the `values` parameter.
Pandas will create a hierarchical column index (MultiIndex) for the new table.

In [6]:
d.pivot(index='Item', columns='CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0     2$   1$    NaN     2€   1€    NaN
Item1    NaN   3$     4$    NaN   3€     4€

## Stack/Unstack

In fact pivoting a table is a special case of stacking a DataFrame:

- **Stacking**: means moving (also pivoting) the __innermost__ column index to become the innermost row index.
- **Unstacking**: means moving the __innermost__ row index to become the innermost column index.

In [7]:
# Row Multi-Index
row_idx_arr = list(zip(['r0', 'r0'], ['r-00', 'r-01']))
row_idx = pd.MultiIndex.from_tuples(row_idx_arr)

# Column Multi-Index
col_idx_arr = list(zip(['c0', 'c0', 'c1'], ['c-00', 'c-01', 'c-10']))
col_idx = pd.MultiIndex.from_tuples(col_idx_arr)

# Create the DataFrame
d3 = DataFrame(np.arange(6).reshape(2,3), index=row_idx, columns=col_idx).applymap(lambda x: (x // 3, x % 3))
d3

c0              c1
           c-00    c-01    c-10
r0 r-00  (0, 0)  (0, 1)  (0, 2)
   r-01  (1, 0)  (1, 1)  (1, 2)

### Stack

In [8]:
d3.stack()  # default (-1), stacking innermost (last) column index

c0      c1
r0 r-00 c-00  (0, 0)     NaN
        c-01  (0, 1)     NaN
        c-10     NaN  (0, 2)
   r-01 c-00  (1, 0)     NaN
        c-01  (1, 1)     NaN
        c-10     NaN  (1, 2)

In [9]:
d3.stack(level=0)

c-00    c-01    c-10
r0 r-00 c0  (0, 0)  (0, 1)     NaN
        c1     NaN     NaN  (0, 2)
   r-01 c0  (1, 0)  (1, 1)     NaN
        c1     NaN     NaN  (1, 2)

### Unstack

In [10]:
d3.unstack()  # default (-1), unstacking innermost (last) row index

c0                              c1        
      c-00            c-01            c-10        
      r-00    r-01    r-00    r-01    r-00    r-01
r0  (0, 0)  (1, 0)  (0, 1)  (1, 1)  (0, 2)  (1, 2)

In [11]:
d3.unstack(0)

c0              c1
        c-00    c-01    c-10
          r0      r0      r0
r-00  (0, 0)  (0, 1)  (0, 2)
r-01  (1, 0)  (1, 1)  (1, 2)

![](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/stack-unstack1.png)

<img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_stack.png" width="640">
<img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_unstack.png" width="640">

### Multiple Levels

In [12]:
columns = pd.MultiIndex.from_tuples([
        ('A', 'cat', 'long'), ('B', 'cat', 'long'),
        ('A', 'dog', 'short'), ('B', 'dog', 'short')
    ],
    names=['exp', 'animal', 'hair_length']
)
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,1.059746,-0.612812,-0.690151,0.317193
1,-0.180945,2.209117,0.634254,1.548406
2,-0.423173,0.076207,0.610957,-0.166565
3,-0.037512,-0.595232,-0.928658,0.267121


In [13]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long         1.059746 -0.612812
  dog    short       -0.690151  0.317193
1 cat    long        -0.180945  2.209117
  dog    short        0.634254  1.548406
2 cat    long        -0.423173  0.076207
  dog    short        0.610957 -0.166565
3 cat    long        -0.037512 -0.595232
  dog    short       -0.928658  0.267121

In [14]:
# from above is equivalent to:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long         1.059746 -0.612812
  dog    short       -0.690151  0.317193
1 cat    long        -0.180945  2.209117
  dog    short        0.634254  1.548406
2 cat    long        -0.423173  0.076207
  dog    short        0.610957 -0.166565
3 cat    long        -0.037512 -0.595232
  dog    short       -0.928658  0.267121

### Fill missing data

- __Unstacking__ can result in __missing values__ if subgroups do not have the same set of labels.
- By default, missing values will be replaced with the default fill value for that data type, `NaN` for float, `NaT` for datetimelike

In [15]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                     ('B', 'cat'), ('A', 'dog')],
                                    names=['exp', 'animal'])

index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                    ('one', 'two')],
                                    names=['first', 'second']) 

df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.072771  0.197562
      two    -2.672011  0.412956
foo   one    -0.775687  0.917038
qux   two    -0.245372 -0.940731

In [16]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.072771 -2.672011  0.197562  0.412956
foo    -0.775687       NaN  0.917038       NaN
qux          NaN -0.245372       NaN -0.940731

In [17]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar    -7.277132e-02 -2.672011e+00  1.975617e-01  4.129559e-01
foo    -7.756872e-01 -1.000000e+09  9.170382e-01 -1.000000e+09
qux    -1.000000e+09 -2.453722e-01 -1.000000e+09 -9.407306e-01

### Calculate

In [18]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.251463 -0.072771  0.197562 -1.342170
      two     1.921913 -2.672011  0.412956  0.231708
baz   one    -0.694452 -0.133343  2.163149  1.480759
      two     0.487188 -1.572656 -1.916679  0.347214
foo   one     0.618454 -0.775687  0.917038 -1.068503
      two    -0.162572 -2.621306  2.388893  0.356138
qux   one    -1.372046 -0.910867  0.003329  2.403540
      two    -0.466882 -0.245372 -0.940731  0.447336

In [19]:
df.stack()

exp                         A         B
first second animal                    
bar   one    cat     0.251463  0.197562
             dog    -1.342170 -0.072771
      two    cat     1.921913  0.412956
             dog     0.231708 -2.672011
baz   one    cat    -0.694452  2.163149
             dog     1.480759 -0.133343
      two    cat     0.487188 -1.916679
             dog     0.347214 -1.572656
foo   one    cat     0.618454  0.917038
             dog    -1.068503 -0.775687
      two    cat    -0.162572  2.388893
             dog     0.356138 -2.621306
qux   one    cat    -1.372046  0.003329
             dog     2.403540 -0.910867
      two    cat    -0.466882 -0.940731
             dog     0.447336 -0.245372

In [20]:
df.stack().mean(1)

first  second  animal
bar    one     cat       0.224512
               dog      -0.707471
       two     cat       1.167435
               dog      -1.220152
baz    one     cat       0.734348
               dog       0.673708
       two     cat      -0.714745
               dog      -0.612721
foo    one     cat       0.767746
               dog      -0.922095
       two     cat       1.113160
               dog      -1.132584
qux    one     cat      -0.684359
               dog       0.746337
       two     cat      -0.703806
               dog       0.100982
dtype: float64

In [21]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     0.224512 -0.707471
      two     1.167435 -1.220152
baz   one     0.734348  0.673708
      two    -0.714745 -0.612721
foo   one     0.767746 -0.922095
      two     1.113160 -1.132584
qux   one    -0.684359  0.746337
      two    -0.703806  0.100982

In [22]:
# same result, another way
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one     0.224512 -0.707471
      two     1.167435 -1.220152
baz   one     0.734348  0.673708
      two    -0.714745 -0.612721
foo   one     0.767746 -0.922095
      two     1.113160 -1.132584
qux   one    -0.684359  0.746337
      two    -0.703806  0.100982

## Reshaping by Melt

In [23]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                       'last' : ['Doe', 'Bo'],
                       'height' : [5.5, 6.0],
                       'weight' : [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [24]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [25]:
# custom variable names
cheese.melt(id_vars=['first', 'last'], var_name='quantity', value_name='measure')

,first,last,quantity,measure
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


<img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_melt.png" width="640">

###  wide_to_long() = 

Panel data convenience function. It is less flexible than melt(), but more user-friendly.

In [26]:
dft = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                    "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                    "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                    "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                    "X"     : dict(zip(range(3), np.random.randn(3)))
                   })
dft["id"] = dft.index
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,0.180674,0
1,b,e,1.2,1.3,1.511357,1
2,c,f,0.7,0.1,-1.579152,2


In [27]:
# The wide format variables are assumed to start with the stub names.
pd.wide_to_long(dft, stubnames=["A", "B"], i="id", j="year", suffix=r'\d+')

,,X,A,B
id,year,,,
0,1970,0.180674,a,2.5
1,1970,1.511357,b,1.2
2,1970,-1.579152,c,0.7
0,1980,0.180674,d,3.2
1,1980,1.511357,e,1.3
2,1980,-1.579152,f,0.1


### Tidy data by hand

In [28]:
(
    dft.melt(id_vars=['X', 'id'], var_name='_')
    .assign(
        var=lambda x: x._.str[0],
        year=lambda x: x._.str[1:])
    .loc[:, ['X', 'id', 'value', 'var', 'year']]
)

,X,id,value,var,year
0,0.180674,0,a,A,1970
1,1.511357,1,b,A,1970
2,-1.579152,2,c,A,1970
3,0.180674,0,d,A,1980
4,1.511357,1,e,A,1980
5,-1.579152,2,f,A,1980
6,0.180674,0,2.5,B,1970
7,1.511357,1,1.2,B,1970
8,-1.579152,2,0.7,B,1970
9,0.180674,0,3.2,B,1980


In [29]:
(
    dft.melt(id_vars=['X', 'id'], var_name='_')
    .assign(
        var=lambda x: x._.str[0],
        year=lambda x: x._.str[1:])
    .loc[:, ['X', 'id', 'value', 'var', 'year']]
    .set_index(['id', 'year', 'X', 'var'])
    .unstack()
    .transpose().reset_index(level=0, drop=True).transpose() # drop column MultiIndex level
    .reset_index()
    .rename_axis(None, axis=1)
)

,id,year,X,A,B
0,0,1970,0.180674,a,2.5
1,0,1980,0.180674,d,3.2
2,1,1970,1.511357,b,1.2
3,1,1980,1.511357,e,1.3
4,2,1970,-1.579152,c,0.7
5,2,1980,-1.579152,f,0.1


In [30]:
# bug: https://github.com/pandas-dev/pandas/issues/21425
(
    dft.melt(id_vars=['X', 'id'], var_name='_')
    .assign(
        var=lambda x: x._.str[0],
        year=lambda x: x._.str[1:])
    .loc[:, ['X', 'id', 'value', 'var', 'year']]
    .pivot_table(index=['id', 'year'], columns='var', values='value')
)

DataError: No numeric types to aggregate

## Pivot tables

While `pivot()` provides general purpose pivoting with various data types (strings, numerics, etc.), pandas also provides `pivot_table()` for pivoting with __aggregation__ of __numeric__ data.

If the `values` column name is not given, the pivot table will include all of the data that can be aggregated in an additional level of hierarchy in the columns:

In [31]:
import datetime
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                   'B': ['A', 'B', 'C'] * 8,
                   'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                   'D': np.random.randn(24),
                   'E': np.random.randn(24),
                   'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                        [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
df.head()

,A,B,C,D,E,F
0,one,A,foo,0.617340,0.688955,2013-01-01
1,one,B,foo,0.114991,0.616782,2013-02-01
2,two,C,foo,-1.732448,0.778316,2013-03-01
3,three,A,bar,-0.963644,0.834542,2013-04-01
4,one,B,bar,0.235077,-1.513572,2013-05-01


In [32]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.869647 -0.039520
      B  0.103991  0.037492
      C -0.647924 -0.274917
three A -0.835807       NaN
      B       NaN -0.006188
      C -0.327788       NaN
two   A       NaN  0.388725
      B -0.207472       NaN
      C       NaN -0.213953

In [33]:
pd.pivot_table(df, values='D', index=['B'], columns=['A', 'C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  1.739294 -0.079040 -1.671613       NaN       NaN  0.777451
B  0.207981  0.074984       NaN -0.012375 -0.414943       NaN
C -1.295847 -0.549834 -0.655576       NaN       NaN -0.427907

In [34]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  1.739294 -0.079040 -1.671613       NaN       NaN  0.777451 -1.447236   
B  0.207981  0.074984       NaN -0.012375 -0.414943       NaN -1.233076   
C -1.295847 -0.549834 -0.655576       NaN       NaN -0.427907 -0.495008   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A -1.468236 -0.607316       NaN       NaN  0.406707  
B  0.126317       NaN -0.239886 -1.678184       NaN  
C  2.150398 -2.353556       NaN       NaN  3.712925

### Add margins

In [35]:
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.876525  0.928941  0.905135  1.892883  2.012529  1.595137
      B  0.185383  0.109601  0.130130  1.268597  0.782942  0.945925
      C  0.527206  1.082905  0.727957  1.241280  0.583491  1.100118
three A  0.180789       NaN  0.180789  1.609658       NaN  1.609658
      B       NaN  1.158156  1.158156       NaN  1.470418  1.470418
      C  0.648783       NaN  0.648783  0.266018       NaN  0.266018
two   A       NaN  1.014202  1.014202       NaN  1.524767  1.524767
      B  0.454298       NaN  0.454298  0.105855       NaN  0.105855
      C       NaN  2.147476  2.147476       NaN  1.524730  1.524730
All      0.703774  0.933042  0.795059  0.982093  1.361359  1.250150